In [ ]:
# https://www.datacamp.com/tutorial/generative-adversarial-networks
# https://colab.research.google.com/drive/1PErY9XmsMTGOYApgh1xmErXkmXbMSNjY

# https://medium.datadriveninvestor.com/generative-adversarial-network-gan-using-keras-ce1c05cfdfd3

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.datasets import mnist
import numpy as np

def load_mnist_dataset():
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train = (x_train.astype(np.float32) - 127.5)/127.5
  # Convert shape from (60000, 28, 28) to (60000, 784)
  x_train = x_train.reshape(60000, 784)
  return (x_train, y_train, x_test, y_test)

(X_train, y_train,X_test, y_test)=load_mnist_dataset()
print(X_train.shape)

11490434/11490434 [==============================] - 0s 0us/step
(60000, 784)


In [ ]:
from keras.optimizers import Adam

optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, LeakyReLU
from keras import initializers

def generator_model(latent_dimension):

  generator = Sequential()
  generator.add(Dense(256, input_dim=latent_dimension))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(512))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(1024))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(784, activation='tanh'))
  generator.compile(loss='binary_crossentropy', optimizer=optimizer)

  return generator

generator = generator_model(100)
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25856     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 1024)              525312    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 784)               8

In [ ]:
def discriminator_model():

  discriminator = Sequential()
  discriminator.add(Dense(1024, input_dim=784))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(512))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(256))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(1, activation='sigmoid'))
  discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)

  return discriminator

discriminator = discriminator_model()
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 1024)              803840    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 1024)              0         
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_5 (Dense)             (None, 512)               524800    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 512)               0         
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 256)              

In [ ]:
from keras.layers import Input

def gan_model(generator, discriminator, latent_dimension):

  discriminator.trainable = False
  # The , after latent_dimension is to indicate to Python that it is a tuple and not a value
  gan_input = Input(shape=(latent_dimension,))

  x = generator(gan_input)
  gan_output = discriminator(x)

  gan = Model(inputs=gan_input, outputs=gan_output)
  gan.compile(loss='binary_crossentropy', optimizer='adam')

  return gan

gan = gan_model(generator, discriminator, 100)
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 784)               1486352   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 1460225   
                                                                 
Total params: 2946577 (11.24 MB)
Trainable params: 1486352 (5.67 MB)
Non-trainable params: 1460225 (5.57 MB)
_________________________________________________________________


In [ ]:
import matplotlib.pyplot as plt
np.random.seed(10)

def plot_generated_images(epoch, generator, latent_dimension, examples=100, dim=(10, 10), figsize=(10, 10)):

  noise = np.random.normal(0, 1, size=[examples, latent_dimension])
  generated_images = generator.predict(noise)
  generated_images = generated_images.reshape(examples, 28, 28)

  plt.figure(figsize=figsize)
  for i in range(generated_images.shape[0]):
    plt.subplot(dim[0], dim[1], i+1)
    plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
    plt.axis('off')

  plt.tight_layout()
  plt.savefig('/content/drive/MyDrive/Colab Notebooks/gan_generated_image_epoch_%d.png' % epoch)

In [ ]:
from tqdm import tqdm

def train(epochs, batch_size):

  X_train, y_train, X_test, y_test = load_mnist_dataset()
  batch_count = X_train.shape[0] / batch_size

  generator = generator_model(latent_dimension=100)
  discriminator = discriminator_model()

  gan = gan_model(generator, discriminator, 100)

  for e in range(1, epochs+1):
    print("Epoch %d" %e)

    for _ in tqdm(range(batch_size)):
      # Generate random noise
      noise= np.random.normal(0,1, [batch_size, 100])
      generated_images = generator.predict(noise)
      # Get a random set of real images
      real_images = X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]

      X= np.concatenate([real_images, generated_images])
      # Initialize the first half of labels as real and rest generated
      y_dis=np.zeros(2*batch_size)
      y_dis[:batch_size]=0.9

      # Pre train discriminator on fake and real data before starting the GAN
      discriminator.trainable = True
      discriminator.train_on_batch(X, y_dis)

      # Staging the noised input of Generator as real data
      noise= np.random.normal(0,1, [batch_size, 100])
      y_gen = np.ones(batch_size)

      # Fix the weights of discriminator for the GAN training
      discriminator.trainable = False
      gan.train_on_batch(noise, y_gen)

      if e == 1 or e % 20 == 0:
        plot_generated_images(e, generator, 100)

In [ ]:
train(400, 128)